In [1]:
"""
experimentation.ipynb

File for experimenting with new things

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

from src.harness.constants import Constants as C
from src.harness.dataset import load_and_process_mnist
from src.harness.experiment import ExperimentData
from src.harness.metrics import get_train_test_loss_accuracy
from src.harness.model import create_lenet_300_100, create_masked_nn, pruned_nn
from src.harness import paths
from src.harness.pruning import create_pruning_callback, create_pruning_parameters, get_layer_weight_counts, get_pruning_percents
from src.harness.training import train, TrainingRound
from src.harness.utils import count_nonzero_parameters

In [2]:
# Constants
DEBUG: bool = True

In [3]:
pruning_params_unpruned: dict = create_pruning_parameters(0.01, 0, 0, 100)
pruning_callback: list = create_pruning_callback('val_loss', 3, 0.001)

In [4]:
train_loss, train_accuracy, test_loss, test_accuracy = get_train_test_loss_accuracy()

In [5]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()

In [6]:
# Create a masked model
pruning_params_unpruned: dict = create_pruning_parameters(0.01, 0, 0, 100)
args: tuple = (0, create_lenet_300_100, pruning_params_unpruned)
model = pruned_nn(*args)
mask_model = create_masked_nn(*args)
layer_weight_counts: list[int] = get_layer_weight_counts(mask_model)
sparsities: list[float] = get_pruning_percents(layer_weight_counts, .2, .01)
num_pruning_rounds: int = len(sparsities)
count_nonzero_parameters(mask_model)

Nonzero parameters in layer 0 synapses: 235200
Nonzero parameters in layer 0 neurons: 300
Nonzero parameters in layer 1 synapses: 30000
Nonzero parameters in layer 1 neurons: 100
Nonzero parameters in layer 2 synapses: 1000
Nonzero parameters in layer 2 neurons: 10
Total nonzero parameters: 266610


In [7]:
model, mask_model, training_round = train(
    0,
    0,
    model,
    mask_model,
    load_and_process_mnist,
    get_train_test_loss_accuracy
)

TypeError: in user code:

    File "/Users/jordan/Projects/lottery-tickets/src/harness/training.py", line 123, in test_step  *
        predictions = model(data)
    File "/Users/jordan/anaconda3/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 565, in error_handler  *
        del filtered_tb
    File "/Users/jordan/anaconda3/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 588, in __call__  *
        return super().__call__(*args, **kwargs)
    File "/Users/jordan/anaconda3/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 560, in error_handler  *
        filtered_tb = _process_traceback_frames(e.__traceback__)
    File "/Users/jordan/anaconda3/lib/python3.11/site-packages/tf_keras/src/engine/base_layer.py", line 1103, in __call__  *
        input_spec.assert_input_compatibility(
    File "/Users/jordan/anaconda3/lib/python3.11/site-packages/tf_keras/src/engine/input_spec.py", line 213, in assert_input_compatibility  *
        rai